# Avaliação do Modelo e Conclusão

Este notebook cobre o ponto 5: métricas de regressão, comparação de modelos e reflexão sobre melhorias.

## Passos desta seção

1. Recarregar métricas sumarizadas e os modelos ajustados.
2. Avaliar os modelos no conjunto de teste e gerar métricas RMSE/MAE/R².
3. Visualizar o benchmark resultante.
4. Propor ajustes futuros e concluir qual modelo entrega melhor trade-off.

In [ ]:
import pandas as pd
import joblib
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

DATA_FILE = Path('data/clean/olist_ml_ready.csv')
metrics_file = Path('src/models/training_metrics.json')
MODEL_DIR = Path('src/models/trained')

df = pd.read_csv(DATA_FILE)
num_rows = len(df)
print(f'Dataset carregado ({num_rows} registros).')


### Dataset e métricas carregados

Recarrega o CSV limpo e prepara o caminho para ler os resultados já calculados (modelos e métricas).

In [ ]:
drop_cols = [
    'order_id', 'customer_id',
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
TARGET = 'delivery_time_days'
X = df.drop(columns=drop_cols + [TARGET], errors='ignore')
y = df[TARGET]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)
print('Split reproduzido. Teste:', X_test.shape)


### Estrutura das features

Remove colunas temporais e identifica `delivery_time_days` como alvo, garantindo que os mesmos atributos usados no treino sejam avaliados.

In [ ]:
if metrics_file.exists():
    saved_metrics = pd.read_json(metrics_file)
    saved_metrics
else:
    print('Arquivo de métricas não encontrado.')


### Métricas salvas

Exibe as métricas registradas pelo script de treino, facilitando a comparação textual antes de gerar o benchmark gráfico.

In [ ]:
benchmark = []
if MODEL_DIR.exists():
    for model_path in MODEL_DIR.glob('*.joblib'):
        name = model_path.stem
        estimator = joblib.load(model_path)
        preds = estimator.predict(X_test)
        benchmark.append({
            'modelo': name,
            'rmse': mean_squared_error(y_test, preds, squared=False),
            'mae': mean_absolute_error(y_test, preds),
            'r2': r2_score(y_test, preds)
        })
benchmark_df = pd.DataFrame(benchmark).sort_values('rmse')
benchmark_df


### Comparação dos modelos

Carrega cada artefato `joblib`, prevê o conjunto de teste e calcula RMSE/MAE/R² para ordenar os candidatos finais.

### Visualização do benchmark

O gráfico abaixo destaca o RMSE absoluto de cada modelo no conjunto de teste, facilitando a escolha final.

In [ ]:
import plotly.express as px

fig = px.bar(
    benchmark_df,
    x='modelo',
    y='rmse',
    title='RMSE no conjunto de teste por modelo',
    text='rmse'
)
fig.update_traces(texttemplate='%{text:.2f}')
fig.show()


### Discussão e próximos passos

- O melhor modelo entrega o menor RMSE e um R² razoável, indicando boa explicabilidade.
- Melhorias futuras: incluir lag temporal (horário da compra), integrar rotas entre cidades, testar modelos de séries temporais ou redes neurais leves.
- Observações de negócio: o modelo pode suportar alertas práticos de atrasos e priorização de entregas.